In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds

print(tf.__version__)
print(tfds.__version__)

imdb, info =tfds.load('imdb_reviews', with_info=True, as_supervised=True)

2.1.0
4.5.2


Dl Completed...: 100%|██████████| 1/1 [00:16<00:00, 16.23s/ url]


Dataset imdb_reviews downloaded and prepared to C:\Users\WX847\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [4]:
?tfds.load

Signature:
tfds.load(
    name: str,
    *,
    split: Union[str, ForwardRef('AbstractSplit'), Any, NoneType] = None,
    data_dir: Union[str, NoneType] = None,
    batch_size: Union[int, NoneType] = None,
    shuffle_files: bool = False,
    download: bool = True,
    as_supervised: bool = False,
    decoders: Union[tensorflow_datasets.core.decode.base.Decoder, ForwardRef('PartialDecoding'), Dict[str, ForwardRef('TreeDict')], NoneType] = None,
    read_config: Union[tensorflow_datasets.core.utils.read_config.ReadConfig, NoneType] = None,
    with_info: bool = False,
    builder_kwargs: Union[Dict[str, Any], NoneType] = None,
    download_and_prepare_kwargs: Union[Dict[str, Any], NoneType] = None,
    as_dataset_kwargs: Union[Dict[str, Any], NoneType] = None,
    try_gcs: bool = False,
)
Docstring:
Loads the named dataset into a `tf.data.Dataset`.

`tfds.load` is a convenience method that:

1. Fetch the `tfds.core.DatasetBuilder` by name:

   ```python
   builder = tfds.builder(name, d

数据预处理

In [6]:
import numpy as np

train_data, test_data = imdb['train'],imdb['test']
training_sentences = []
training_labels = []

testing_sentences = []  
testing_labels = []

for s,l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())
  
  
for s,l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())
 
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [16]:
?s

Type:            EagerTensor
String form:     tf.Tensor(b"They just don't make cartoons like they used to. This one had wit, great characters,  <...> artoon shows, and one of the most honored, winning several Emmy Awards.", shape=(), dtype=string)
File:            c:\users\wx847\anaconda3\envs\learn_tensorflow\lib\site-packages\tensorflow_core\python\framework\ops.py
Docstring:       <no docstring>
Class docstring: Base class for EagerTensor.


In [17]:
?l

Type:            EagerTensor
String form:     tf.Tensor(1, shape=(), dtype=int64)
File:            c:\users\wx847\anaconda3\envs\learn_tensorflow\lib\site-packages\tensorflow_core\python\framework\ops.py
Docstring:       <no docstring>
Class docstring: Base class for EagerTensor.


In [21]:
testing_labels_final.shape

(25000,)

In [23]:

trunc_type = 'post'  # 截断方式，后边的被丢弃
oov_tok = '<OOV>'  # 未登录词语

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 训练集，进行词条化，然后进行序列化
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length,truncating=trunc_type,padding='post')

# 测试集，用训练集的tokenizer词条化, 然后进行序列化即可
testing_sequences= tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length)



检查一下编的对不对

In [33]:
reverse_word_index = {v:k for k,v in word_index.items()}  # 反转

def decode_review(text):
    return ' '.join([reverse_word_index.get(i,'?') for i in text])
print(decode_review(padded[1]))
print(training_sentences[1])


b'i have been known to fall asleep during films but this is usually due to a combination of things including really tired being warm and comfortable on the <OOV> and having just eaten a lot however on this occasion i fell asleep because the film was rubbish the plot development was constant constantly slow and boring things seemed to happen but with no explanation of what was causing them or why i admit i may have missed part of the film but i watched the majority of it and everything just seemed to happen of its own <OOV> without any real concern for anything else i cant recommend this film at all ' ? ? ? ? ? ? ?
b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of 

设置嵌入模型

In [35]:
vocab_size = 10000   # 最大的词语数量
embedding_dim = 16  # 每个word都embedding为一个长度为16的向量
max_length = 120  # 句子的最长

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


训练模型

In [37]:
num_epochs = 10
# 将上边处理好的序列，添加到模型进行训练
model.fit(padded,training_labels_final,epochs=num_epochs, validation_data=(testing_padded, testing_labels_final)) 


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 7s 261us/sample - loss: 0.4879 - accuracy: 0.7513 - val_loss: 0.3621 - val_accuracy: 0.8396
Epoch 2/10
25000/25000 [==============================] - 5s 198us/sample - loss: 0.2464 - accuracy: 0.9042 - val_loss: 0.3726 - val_accuracy: 0.8390
Epoch 3/10
25000/25000 [==============================] - 5s 195us/sample - loss: 0.1051 - accuracy: 0.9720 - val_loss: 0.4636 - val_accuracy: 0.8206
Epoch 4/10
25000/25000 [==============================] - 5s 215us/sample - loss: 0.0284 - accuracy: 0.9957 - val_loss: 0.5510 - val_accuracy: 0.8126
Epoch 5/10
25000/25000 [==============================] - 5s 204us/sample - loss: 0.0071 - accuracy: 0.9996 - val_loss: 0.6144 - val_accuracy: 0.8188
Epoch 6/10
25000/25000 [==============================] - 5s 200us/sample - loss: 0.0023 - accuracy: 0.9999 - val_loss: 0.6764 - val_accuracy: 0.8180
Epoch 7/10
25000/25000 [==========================

查看第一层embedding的维度

In [38]:
e =model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)  # shape: (vocab_size, embedding_dim)  # 词汇表的大小，每个word都embedding为一个长度为16的向量

(10000, 16)
